# Benchmark modelos de Transformers (BETO-Galén) para obtener Embeddings del texto

In [ ]:
! python -m spacy download es_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-27 11:47:59.356263: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
! pip install spacy
#! python -m spacy download es
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def NLP_preprocesaDocumento(nlp, doc):

  #Convertir a minúsculas
  doc = doc.lower()
  
  #Tokenizado
  doc = [" ".join([token.text for token in nlp(doc)])][0]

  #Eliminación de signos de puntuación
  doc = [" ".join([token.lemma_ for token in nlp(doc) if not token.is_punct]) ][0]

  #Eliminación de stopwords
  spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
  doc = [" ".join([token for token in doc.split() if not token.lower() in spacy_stopwords])][0]
  
  #Lematización de palabras
  doc = [" ".join([token.lemma_ for token in nlp(doc)])][0]

  #Eliminación de tildes
  tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'ü': 'u', 'Ü': 'U'}
  for tilde in tildes:
	  if tilde in doc:
		  doc = doc.replace(tilde, tildes[tilde])

  return doc


In [ ]:
def split_sentences(nlp, text):
    tokens = nlp(text)
    sentences = []
    for sent in tokens.sents:
        sentences.append(sent.text.strip())
    return sentences

In [ ]:
#Cargamos el modelo en español
nlp_lg = spacy.load("es_core_news_lg")

## Carga de las terminologías clínicas SNOMED-CT y generación de sus Embeddings
NOTA: Estos solo es necesario hacerlo la primera vez

In [ ]:
import os
def cargaProblemasSaludAHSpainExtension():
  f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
  terminos_snomedct_problemassaludAH = []
  cont = 0
  for row in f:
    if cont>0:
      values = row.strip().split("\t")
      #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
      #print(values)
      termino = {}
      termino["id"] = values[0];
      #termino["effectiveTime"] = values[1];
      #termino["active"] = values[2];
      #termino["moduleId"] = values[3];
      #termino["refsetId"] = values[4];
      #termino["referencedComponentId"] = values[5];
      termino["term"] = values[6];
      #termino["num_palabras"] = len(values[6].split(" "));
      #print(termino)
      terminos_snomedct_problemassaludAH.append(termino)
    cont = cont + 1

  f.close();
  return terminos_snomedct_problemassaludAH;

In [ ]:
def anadirEmbedding(model,nlp,lista,num_items_max=0,preprocesado=False):
  cont = 0
  for item in lista:
    #print(item["term"])
    if preprocesado:
      doc = NLP_preprocesaDocumento(nlp,item["term"])
      if doc == "":
        print("ERROR AL Parsear " + item["term"])
        doc = item["term"]
    else:
      doc = item["term"]
    #print(doc)
    #item["embedding"] = model.wv.get_vector(doc)

    tokens = tokenizer.tokenize(doc)
    input_ids = get_ids(doc, tokens, tokenizer, SEQ_LEN)
    input_masks = get_masks(tokens,SEQ_LEN)
    input_segments = get_segments(tokens,SEQ_LEN)
    input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
    input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
    input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
    embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
    item["embedding"] = np.mean(embeddings[0],axis=0)

    cont = cont + 1
    if cont == num_items_max:
      break
  return lista

In [ ]:
terminos_snomedct_problemassaludAH = cargaProblemasSaludAHSpainExtension();

In [ ]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

#numpyArrayOne = numpy.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]])

# Serialization
#numpyData = {"array": numpyArrayOne}
#encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
#print("Printing JSON serialized NumPy array")
#print(encodedNumpyData)

# Deserialization
#print("Decode JSON serialized NumPy array")
#decodedArrays = json.loads(encodedNumpyData)

#finalNumpyArray = numpy.asarray(decodedArrays["array"])
#print("NumPy Array")
#print(finalNumpyArray)

In [ ]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
import numpy as np
#from nlp_utils import *

# BETO tokenizer
!pip install keras_bert
from keras_bert import load_vocabulary, Tokenizer
model_path = "drive/MyDrive/models/BETO_Galen/"
#https://github.com/dccuchile/beto/blob/master/config/uncased_2M/config.json
config_path = model_path + "config.json"
#https://github.com/guilopgar/ClinicalCodingTransformerES/tree/main/BETO/BETO-Galen
checkpoint_path = "drive/MyDrive/models/BETO_Galen/model.ckpt-1000000"
#https://github.com/dccuchile/beto/blob/master/config/uncased_2M/vocab.txt
vocab_file = "vocab.txt"
tokenizer = Tokenizer(token_dict=load_vocabulary(model_path + vocab_file), pad_index=1, cased=True)

# Hyper-parameters
training = False
trainable = True
SEQ_LEN = 128

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(text, tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    #token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids = tokenizer.encode(text)[0]
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

Obtención de embeddings con el modelo seleccionado (nlp_lg y model BETO-Galén)

In [ ]:
from keras_bert import load_trained_model_from_checkpoint
#Tiempo de ejecucion: 60 sg
model = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)

In [ ]:
#Tiempo de ejecucion: 40 min (500ms/entrada)
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

1/1 [==============================] - 1s 515ms/step
ERROR AL Parsear estado de gran mal
1/1 [==============================] - 1s 554ms/step


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_beto_galen.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

## Obtención de embeddings, carga de modelos y pruebas

In [ ]:
def obtenEmbedding(model,doc):
    tokens = tokenizer.tokenize(doc)
    input_ids = get_ids(doc, tokens, tokenizer, SEQ_LEN)
    input_masks = get_masks(tokens,SEQ_LEN)
    input_segments = get_segments(tokens,SEQ_LEN)
    input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
    input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
    input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
    embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
    return np.mean(embeddings[0],axis=0)

In [ ]:
from keras_bert import load_trained_model_from_checkpoint
#Tiempo de ejecucion: 60 sg
model_beto_galen = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)


In [ ]:
import os, json, numpy

f = open('catalogo_beto_galen.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_beto_galen = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_beto_galen.append(termino)
f.close();


In [ ]:
def NLP_get_ngrams(text, n):
    text = text.split()
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

In [ ]:
#informe = "El paciente presenta el abdomen distenido."
#informe = "El paciente presenta un aborto espontáneo y retenido, distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
#informe = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."


In [ ]:
import numpy as np
from numpy.linalg import norm
from operator import itemgetter

def get_terminos_vinculados(model, catalog, similarity_threshold, similarity, informe):
  terminos_vinculados = []

  frases = split_sentences(nlp_lg, informe)
  for frase in frases:
    informe_preprocesado = NLP_preprocesaDocumento(nlp_lg, frase)

    #Empezamos por ngramas más grandes a más pequeños
    #for num_ngrams in range(3, 0, -1):
      #ngramas = NLP_get_ngrams(informe_preprocesado, num_ngrams)
      #for ngrama in ngramas:
        #doc = " ".join([token for token in ngrama])
        #print(doc)
    doc = informe_preprocesado
    doc_embedding = obtenEmbedding(model,doc)
    vector_A = doc_embedding
    #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
    terminos_candidatos = []
    for termino in catalog:
      vector_B = termino["embedding"]
      if similarity == 0:
        dist = numpy.linalg.norm(vector_A-vector_B)
        #print(doc, " ", termino["term"], " Similarity:", dist)
        if dist <= similarity_threshold:
          #print(doc, " ", termino["term"], " Similarity:", dist)
          terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
        #Nos quedamos con el término con menor distancia de los obtenidos
        if terminos_candidatos:
          terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=False) 
          terminos_vinculados.append(terminos_candidatos_sorted[0])
      else:
        dist = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
        #print(doc, " ", termino["term"], " Similarity:", dist)
        if dist >= similarity_threshold:
          #print(doc, " ", termino["term"], " Similarity:", dist)
          terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
        #Nos quedamos con el término con mayor probabilidad de los obtenidos
        if terminos_candidatos:
          terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=True) 
          terminos_vinculados.append(terminos_candidatos_sorted[0])

  print("Resultado:")

  if similarity == 0:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "100.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]>termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  else:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "0.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]<termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  return terminos_vinculados_deduplicated


In [ ]:
#Tiempo de ejecucion: 16 sg
terminos_vinculados = get_terminos_vinculados(model_beto_galen, terminos_snomedct_problemassaludAH_beto_galen, 5, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])


1/1 [==============================] - 1s 556ms/step
Resultado:
abuso de alcohol sin adicción, en remisión  Similarity: 4.423754970487724
aterosclerosis de arteria renal  Similarity: 4.832569314368096
celulitis y absceso de dedo de la mano  Similarity: 4.9732656262201065
coagulación intravascular diseminada  Similarity: 4.908441964808002
convulsión parcial simple sin alteración de la consciencia  Similarity: 4.901772692877756
demencia degenerativa primaria de tipo Alzheimer, de inicio en la etapa senil  Similarity: 4.921527315223315
depresión mayor grave, un solo episodio, sin síntomas psicóticos  Similarity: 4.6592067783074915
derrame pleural  Similarity: 0.0
desplazamiento de disco intervertebral lumbar, sin mielopatía  Similarity: 4.913172374955011
desplazamiento de disco intervertebral, sin mielopatía  Similarity: 4.740609369558047
desviación congénita del tabique nasal  Similarity: 4.790878161931095
enfermedad pulmonar obstructiva crónica con infección aguda de las vías respirator

In [ ]:
#Tiempo de ejecucion: 19 sg
terminos_vinculados = get_terminos_vinculados(model_beto_galen, terminos_snomedct_problemassaludAH_beto_galen, 0.94, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

1/1 [==============================] - 1s 514ms/step
Resultado:
abuso de alcohol sin adicción, en remisión  Cosine Similarity: 0.9542393751694246
aterosclerosis de arteria renal  Cosine Similarity: 0.9452621230248378
celulitis y absceso de dedo de la mano  Cosine Similarity: 0.9436970101233054
coagulación intravascular diseminada  Cosine Similarity: 0.9433295837898372
convulsión parcial simple sin alteración de la consciencia  Cosine Similarity: 0.9452082673707255
demencia degenerativa primaria de tipo Alzheimer, de inicio en la etapa senil  Cosine Similarity: 0.9409228726186352
depresión mayor grave, un solo episodio, sin síntomas psicóticos  Cosine Similarity: 0.9479533261496412
derrame pleural  Cosine Similarity: 0.9999999460310858
desplazamiento de disco intervertebral lumbar, sin mielopatía  Cosine Similarity: 0.9430127277840181
desplazamiento de disco intervertebral, sin mielopatía  Cosine Similarity: 0.9474144141461677
desviación congénita del tabique nasal  Cosine Similarity: 0